# À propos de ce livret

Explication de la méthode *Bayesian temporal matrix factorization*
- What the proposed Bayesian temporal matrix factorization (BTMF for short) is?
- How to implement BTMF mainly using Python `Numpy` with high efficiency?
- How to make data imputations with real-world spatiotemporal datasets?



Recherches liées au développement de cette méthode : 
>Xinyu Chen, Lijun Sun (2019). **Bayesian temporal factorization for multidimensional time series prediction**.


La méthode implantés provient de ces sources : 
>[**Dépôt 1**](https://github.com/xinychen/transdim/) | *Version en python*



**Note sur le cachier**
- Améliorer le temps computationnel


## Préparation pour l'utilisation

In [1]:
# Utilités
import os
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from scipy.stats import invwishart
from numpy.linalg import inv as inv
import time

# Chargement des données
import scipy.io

# Barre de progression
from tqdm.auto import trange
from tqdm import tqdm_notebook, notebook

# Présentation des résultats
import matplotlib.pyplot as plt

Mettre à jour le dossier de travail

In [2]:
os.chdir("C:/Users/amass/OneDrive/02_Education/02_Maitrise/Cours/PROJET_MAITRISE/application")

# Données

## Chargement des données

In [3]:
# Chargement du tenseur de données
tensor = scipy.io.loadmat('data/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']

# Chargement de la matrice aléatoire de données
random_matrix = scipy.io.loadmat('data/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

# Chargement du tenseur aléatoire de données
random_tensor = scipy.io.loadmat('data/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

print("Tensor shape : {:} | Random matrix shape : {:} | Random tensor shape : {:}".format(tensor.shape, 
                                                                                        random_matrix.shape,
                                                                                        random_tensor.shape))

Tensor shape : (214, 61, 144) | Random matrix shape : (214, 61) | Random tensor shape : (214, 61, 144)


## Format des données

In [4]:
petit_donnees = True

# Paramètres du jeu de données tronqué
capteurs = 15
jours = 5
sequences = tensor.shape[2]

matrice = True

if petit_donnees == True:
    tensor = tensor[0:capteurs, 0:jours, 0:sequences]
    
    random_tensor = random_tensor[0:capteurs, 0:jours, 0:sequences]
    

print("The tensors shapes are : {:} | {:}".format(tensor.shape, random_tensor.shape))

if matrice == True:
    dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
    
    print("The matrix shape is : {:}".format(dense_mat.shape))

The tensors shapes are : (15, 5, 144) | (15, 5, 144)
The matrix shape is : (15, 720)


## Scénario de manquants

In [5]:
missing_rate = 0.4

random_missing = True

# Revoir le code pour que les deux choix puissent être respectés sur l'ensemble des matrices/ tenseurs
if random_missing == True:
    print("Random missing (RM) scenario")
    binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
                  .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
    print("\nBinary matrix shape : {:}".format(binary_mat.shape))
else:
    print("Non-random missing (NM) scenario")
    binary_tensor = np.zeros(tensor.shape)
    for i1 in range(tensor.shape[0]):
        for i2 in range(tensor.shape[1]):
            binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
    binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
    print("\nBinary matrix shape : {:}".format(binary_mat.shape))

sparse_mat = np.multiply(dense_mat, binary_mat)

Random missing (RM) scenario

Binary matrix shape : (15, 720)


# Modélisation

## 2) Khatri-Rao product (`kr_prod`)

In [6]:
def kr_prod(a, b):
    """
    Khatri-Rao product
    
    Paramètres en entrée 
        a : Matrice A
        b : Matrice B
    
    Paramètres en sortie
        Produit column-wise (KR roduct)
    
    """
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

## Computing Covariance Matrix (`cov_mat`)


In [7]:
def cov_mat(mat):
    """
    Matrice de covariance
    
    Paramètres en entrée 
        mat : Matrice
    
    Paramètres en sortie 
        new_mat : Matrice de covariance
    
    """
    dim1, dim2 = mat.shape
    new_mat = np.zeros((dim2, dim2))
    mat_bar = np.mean(mat, axis = 0)
    for i in range(dim1):
        new_mat += np.einsum('i, j -> ij', mat[i, :] - mat_bar, mat[i, :] - mat_bar)
    return new_mat

## Tensor Unfolding (`ten2mat`) and Matrix Folding (`mat2ten`)

Using numpy reshape to perform 3rd rank tensor unfold operation. [[**link**](https://stackoverflow.com/questions/49970141/using-numpy-reshape-to-perform-3rd-rank-tensor-unfold-operation)]

In [8]:
def ten2mat(tensor, mode):
    """
    Tensor unfolding
    
    Paramètres en entrée 
        tensor : Tenseur
        mode : 0 (), 1 (), 2 ()
        
    Paramètres en sortie
        Aucun (?)
    
    
    """
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [9]:
def mat2ten(mat, tensor_size, mode):
    """
    Matrix unfolding
    
    Paramètres en entrée 
        mat : Matrice
        tensor_size : Tenseur
        mode : 0 (), 1 (), 2 ()
        
    Paramètres en sortie
        Aucun (?)
    
    
    """
    index = list()
    index.append(mode)
    for i in notebook.tqdm(range(tensor_size.shape[0])):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(tensor_size[index]), order = 'F'), 0, mode)

## Generating Matrix Normal Distributed Random Matrix

In [10]:
def mnrnd(M, U, V):
    """
    Generate matrix normal distributed random matrix.
    
    Paramètes en entrée
        M : m-by-n matrix
        U : m-by-m matrix
        V : n-by-n matrix
    Paramètres en sortie 
        Aucun (?)
    """
    dim1, dim2 = M.shape
    X0 = np.random.rand(dim1, dim2)
    P = np.linalg.cholesky(U)
    Q = np.linalg.cholesky(V)
    return M + np.matmul(np.matmul(P, X0), Q.T)

## Bayesian Temporal Matrix Factorization (BTMF)



In [11]:
def BTMF(dense_mat, sparse_mat, init, rank, time_lags, maxiter1, maxiter2):
    """
    Bayesian Temporal Matrix Factorization, BTMF.
    
    Paramètres en entrée
        dense_mat : Matrice pleine
        sparse_mat : Matrice avec valeurs manquantes
        init : ?
        rank : rangs des facteurs latents
        time_lags : ?
        maxiter1 : ?
        maxiter2 : ?
    
    
    Paramètres en sortie
        mat_hat : Matrice avec valeurs imputées
        W : Facteur latent W
        X_new : ?
        A : ?
    
    
    """
    # Variables bidons
    lrmse = []
    lmape = []
    
    
    # initialisation des paramètres
    W = init["W"]
    X = init["X"]
    
    d = time_lags.shape[0]
    dim1, dim2 = sparse_mat.shape
    pos = np.where((dense_mat != 0) & (sparse_mat == 0))
    position = np.where(sparse_mat != 0)
    binary_mat = np.zeros((dim1, dim2))
    binary_mat[position] = 1
    
    beta0 = 1
    nu0 = rank
    mu0 = np.zeros((rank))
    W0 = np.eye(rank)
    tau = 1
    alpha = 1e-6
    beta = 1e-6
    S0 = np.eye(rank)
    Psi0 = np.eye(rank * d)
    M0 = np.zeros((rank * d, rank))
    
    W_plus = np.zeros((dim1, rank))
    X_plus = np.zeros((dim2, rank))
    X_new_plus = np.zeros((dim2 + 1, rank))
    A_plus = np.zeros((rank, rank, d))
    mat_hat_plus = np.zeros((dim1, dim2 + 1))
    
    # Mise à jour des paramètres
    for iters in notebook.tqdm(range(maxiter1), "Itérations"):
        # Initialisation des paramètres (?)
        W_bar = np.mean(W, axis = 0)
        var_mu_hyper = (dim1 * W_bar)/(dim1 + beta0)
        var_W_hyper = inv(inv(W0) + cov_mat(W) + dim1 * beta0/(dim1 + beta0) * np.outer(W_bar, W_bar))
        var_Lambda_hyper = wishart(df = dim1 + nu0, scale = var_W_hyper, seed = None).rvs()
        var_mu_hyper = mvnrnd(var_mu_hyper, inv((dim1 + beta0) * var_Lambda_hyper))
        
        # ?
        var1 = X.T
        var2 = kr_prod(var1, var1)
        var3 = tau * np.matmul(var2, binary_mat.T).reshape([rank, rank, dim1]) + np.dstack([var_Lambda_hyper] * dim1)
        var4 = (tau * np.matmul(var1, sparse_mat.T)
                + np.dstack([np.matmul(var_Lambda_hyper, var_mu_hyper)] * dim1)[0, :, :])
        
        # Mise à jour de la dimension 1
        for i in notebook.tqdm(range(dim1), desc = "Dim1"):
            inv_var_Lambda = inv(var3[:, :, i])
            W[i, :] = mvnrnd(np.matmul(inv_var_Lambda, var4[:, i]), inv_var_Lambda)
        if iters + 1 > maxiter1 - maxiter2:
            W_plus += W
        
        Z_mat = X[np.max(time_lags) : dim2, :]
        Q_mat = np.zeros((dim2 - np.max(time_lags), rank * d))
        
        # Mise à jour de la dimension 2 (selon le temps)
        for t in notebook.tqdm(range(np.max(time_lags), dim2), "Dimension 2 - temps"):
            Q_mat[t - np.max(time_lags), :] = X[t - time_lags, :].reshape([rank * d])
        
        #
        var_Psi = inv(inv(Psi0) + 
                      np.matmul(Q_mat.T, Q_mat))
        var_M = np.matmul(var_Psi, np.matmul(inv(Psi0), M0) + 
                          np.matmul(Q_mat.T, Z_mat))
        var_S = (S0 + 
                 np.matmul(Z_mat.T,
                           Z_mat) + 
                 np.matmul(np.matmul(M0.T,
                                     inv(Psi0)), 
                           M0) - 
                 np.matmul(np.matmul(var_M.T,
                                     inv(var_Psi)), 
                           var_M))
        Sigma = invwishart(df = nu0 + dim2 - np.max(time_lags), 
                           scale = var_S, 
                           seed = None).rvs()
        
        A = mat2ten(mnrnd(var_M, 
                          var_Psi, 
                          Sigma).T, 
                    np.array([rank, rank, d]), 0)
        
        # ?
        if iters + 1 > maxiter1 - maxiter2:
            A_plus += A
        
        # ?
        Lambda_x = inv(Sigma)
        var1 = W.T
        var2 = kr_prod(var1, var1)
        var3 = tau * np.matmul(var2, binary_mat).reshape([rank, rank, dim2]) + np.dstack([Lambda_x] * dim2)
        var4 = tau * np.matmul(var1, sparse_mat)
        
        # Mise à jour de la dimension 2
        for t in notebook.tqdm(range(dim2), desc = "Dimension 2"):
            Mt = np.zeros((rank, rank))
            Nt = np.zeros(rank)
            # ?
            if t < np.max(time_lags):
                Qt = np.zeros(rank)
            else:
                Qt = np.matmul(Lambda_x, 
                               np.matmul(ten2mat(A, 0), 
                                         X[t - time_lags, :].reshape([rank * d])))
            # ?
            if t < dim2 - np.min(time_lags):
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(np.where((t + time_lags >= np.max(time_lags)) & (t + time_lags < dim2)))[0]
                # ?
                for k in notebook.tqdm(index):
                    Ak = A[:, :, k]
                    Mt += np.matmul(np.matmul(Ak.T, Lambda_x), Ak)
                    A0 = A.copy()
                    A0[:, :, k] = 0
                    var5 = (X[t + time_lags[k], :] 
                            - np.matmul(ten2mat(A0, 0),
                                        X[t + time_lags[k] - time_lags, :].reshape([rank * d])))
                    Nt += np.matmul(np.matmul(Ak.T, Lambda_x),
                                    var5)
            # ?
            var_mu = var4[:, t] + Nt + Qt
            
            # ? 
            if t < np.max(time_lags):
                inv_var_Lambda = inv(var3[:, :, t] + 
                                     Mt - 
                                     Lambda_x + 
                                     np.eye(rank))
            else:
                inv_var_Lambda = inv(var3[:, :, t] + 
                                     Mt)
            
            X[t, :] = mvnrnd(np.matmul(inv_var_Lambda,
                                       var_mu), 
                             inv_var_Lambda)
        
        # Matrice des valeurs imputées (mise à jour)
        mat_hat = np.matmul(W, 
                            X.T)
        # ??
        X_new = np.zeros((dim2 + 1, rank))
        
        # ?
        if iters + 1 > maxiter1 - maxiter2:
            X_new[0 : dim2, :] = X.copy()
            X_new[dim2, :] = np.matmul(ten2mat(A, 0), 
                                       X_new[dim2 - time_lags, :].reshape([rank * d]))
            X_new_plus += X_new
            mat_hat_plus += np.matmul(W, 
                                      X_new.T)
        
        # ?
        tau = np.random.gamma(alpha + 0.5 * sparse_mat[position].shape[0], 
                              1/(beta + 0.5 * np.sum((sparse_mat - mat_hat)[position] ** 2)))
        
        # Calcul du RMSE
        rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2)/dense_mat[pos].shape[0])
        mape = np.sum(np.abs(dense_mat[pos] - mat_hat[pos])/dense_mat[pos])/dense_mat[pos].shape[0]
        lrmse.append(rmse)
        lmape.append(mape)
        
        if (iters + 1) % 200 == 0 and iters < maxiter1 - maxiter2:
            print('Iter: {}'.format(iters + 1))
            print('RMSE: {:.6}'.format(rmse))
            print()
    # ?
    W = W_plus/maxiter2
    X_new = X_new_plus/maxiter2
    A = A_plus/maxiter2
    mat_hat = mat_hat_plus/maxiter2
    
    # Calcul des résultats finaux
    if maxiter1 >= 100:
        final_mape = np.sum(np.abs(dense_mat[pos] - mat_hat[pos])/dense_mat[pos])/dense_mat[pos].shape[0]
        final_rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2)/dense_mat[pos].shape[0])
        
        lrmse.append(final_rmse)
        lmape.append(final_mape)
        
        print('Imputation MAPE: {:.6}'.format(final_mape))
        print('Imputation RMSE: {:.6}'.format(final_rmse))
        print()
    
    return mat_hat, W, X_new, A, lrmse, lmape

## Entrainement du modèle

In [12]:
# Initialisation des paramètres
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 200
maxiter2 = 100
mat_hat, W, X_new, A, lrmse, lmape = BTMF(dense_mat, sparse_mat, init, rank, time_lags, maxiter1, maxiter2)

KeyboardInterrupt: 

# Présentation des résultats

In [ ]:
import matplotlib.pyplot as plt
plt.plot(lrmse)

In [ ]:
plt.plot(lmape)